In [1]:
friend_graph = sc.textFile('/FileStore/tables/soc_LiveJournal1Adj-2d179.txt')

In [2]:
# now we will create rdd of type (key, [connections])

In [3]:
def create_friends_rdd(x):
  x = x.split('\t')
  key = x[0]
  connections = x[1].split(',')
  return (key, connections)

In [4]:
friend_graph = friend_graph.map(create_friends_rdd)

In [5]:
'''
Now we will create MR1
map will create rdd of ((f1,f2),1 or 0)
reducer will reduce this rdd ((f1,f2),#no. of common friends)
'''

Out[ 5 ]: '\nNow we will create MR1\nmap will create rdd of ((f1,f2),1 or 0)\nreducer will reduce this rdd ((f1,f2),#no. of common friends)\n'

In [6]:
def map_pair_friend(x):
  result = list()
  for f1 in x[1]:
    '''
    adding pair person itself to friend
    with no. of friend in commmon to 0
    '''
    result.append(((x[0],f1), 0))
    for f2 in x[1]:
      if f1 != f2:
        # indicates that f1 and f2 are 2 diffferent people that have 1 friennd in common
        result.append(((f1,f2), 1))
  return result   

In [7]:
pair_rdd = friend_graph.flatMap(map_pair_friend)

In [8]:
pair_rdd.take(2)

Out[ 6 ]: [(('0', '1'), 0), (('1', '2'), 1)]

In [9]:
'''
Now reducer will add of all common friends
between f1 and f2 only if f1 is not friend
of f2
'''

In [10]:
def reduce_common_friend_count(x,y):
  if x*y==0:
    return 0
  else:
    return x+y

In [11]:
common_friend_count = pair_rdd.reduceByKey(reduce_common_friend_count)

In [12]:
common_friend_count.take(2)

Out[ 9 ]: [(('4169', '17218'), 1), (('48833', '48856'), 1)]

In [13]:
# filtering out the friends pair
common_friend_count = common_friend_count.filter(lambda x: x[1]!=0)

In [14]:
common_friend_count.cache()

Out[ 11 ]: PythonRDD[8] at RDD at PythonRDD.scala:56

In [15]:
'''
MR2
map - convert RDD of ((f1,f2), c1) to (f1, [(f2,c1)])
reduce - will reduce all the tuple with key as f1 
will give output like (f1 , [(f2,c1), (f3,c2)])
'''

In [16]:
def mapper2(x):
  return (x[0][0], [(x[0][1], x[1])])

In [17]:
friend_recom = common_friend_count.map(mapper2)

In [18]:
friend_recom.take(2)

Out[ 35 ]: [('5682', [('7992', 1)]), ('38575', [('36959', 2)])]

In [19]:
'''
sorting the connection of each
in descending order of number of common friends
'''

In [20]:
friend_recom = friend_recom.sortBy(lambda x: -x[1][0][1])

In [21]:
friend_recom.take(2)

Out[ 38 ]: [('18740', [('18739', 100)]), ('18739', [('18740', 100)])]

In [22]:
friend_recom.cache()

Out[ 39 ]: PythonRDD[36] at RDD at PythonRDD.scala:56

In [23]:
# reducer will combine value of all the keys

In [24]:
def combine_recommended_friends(x,y):
  return x+y

In [25]:
friend_recom = friend_recom.reduceByKey(combine_recommended_friends)

In [26]:
friend_recom.take(2)

Out[ 42 ]: 
[('27479',
 [('27492', 2),
 ('33010', 2),
 ('41807', 2),
 ('27518', 2),
 ('27501', 2),
 ('27531', 2),
 ('27533', 2),
 ('27486', 2),
 ('27524', 2),
 ('27493', 2),
 ('8052', 2),
 ('13074', 2),
 ('27517', 2),
 ('43142', 2),
 ('18276', 2),
 ('21873', 2),
 ('27500', 2),
 ('27504', 2),
 ('27508', 2),
 ('27534', 2),
 ('1673', 2),
 ('27496', 2),
 ('27484', 2),
 ('27510', 2),
 ('27502', 2),
 ('27497', 2),
 ('27499', 2),
 ('27488', 2),
 ('27485', 2),
 ('7611', 2),
 ('27526', 2),
 ('29461', 2),
 ('1664', 2),
 ('27519', 2),
 ('33051', 2),
 ('27511', 2),
 ('20569', 2),
 ('27506', 2),
 ('27490', 2),
 ('27514', 2),
 ('27487', 2),
 ('27512', 2),
 ('27505', 2),
 ('27528', 2),
 ('27529', 2),
 ('27491', 2),
 ('27507', 2),
 ('20826', 2),
 ('27513', 2),
 ('27498', 2),
 ('9659', 2),
 ('25884', 2),
 ('27494', 2),
 ('27515', 2),
 ('27509', 2),
 ('161', 1),
 ('8680', 1),
 ('13462', 1),
 ('49469', 1),
 ('10240', 1),
 ('34026', 1),
 ('37615', 1),
 ('17308', 1),
 ('24051', 1),
 ('6483', 1),
 ('5010', 1),
 ('6200', 1),
 ('38011', 1),
 ('13847', 1),
 ('35585', 1),
 ('22307', 1),
 ('20761', 1),
 ('43162', 1),
 ('20742', 1),
 ('20', 1),
 ('8532', 1),
 ('25186', 1),
 ('14770', 1),
 ('25170', 1),
 ('1555', 1),
 ('7209', 1),
 ('30141', 1),
 ('43354', 1),
 ('1666', 1),
 ('24530', 1),
 ('25257', 1),
 ('11707', 1),
 ('17235', 1),
 ('27521', 1),
 ('8514', 1),
 ('29516', 1),
 ('27527', 1),
 ('13764', 1),
 ('14030', 1),
 ('1692', 1),
 ('44120', 1),
 ('35617', 1),
 ('43626', 1),
 ('25200', 1),
 ('14861', 1),
 ('43622', 1),
 ('22118', 1),
 ('27006', 1),
 ('15969', 1),
 ('16722', 1),
 ('29721', 1),
 ('1670', 1),
 ('9397', 1),
 ('19', 1),
 ('43607', 1),
 ('41456', 1),
 ('44256', 1),
 ('19664', 1),
 ('44291', 1),
 ('44304', 1),
 ('38737', 1),
 ('14332', 1),
 ('22265', 1),
 ('4240', 1),
 ('2640', 1),
 ('27516', 1),
 ('4241', 1),
 ('27483', 1),
 ('16621', 1),
 ('43425', 1),
 ('47770', 1),
 ('24585', 1),
 ('10203', 1),
 ('442', 1),
 ('29444', 1),
 ('31347', 1),
 ('41591', 1),
 ('10120', 1),
 ('16684', 1),
 ('44887', 1),
 ('27584', 1),
 ('43189', 1),
 ('34904', 1),
 ('20290', 1),
 ('32779', 1),
 ('15360', 1),
 ('27808', 1),
 ('14335', 1),
 ('27495', 1),
 ('27523', 1),
 ('9875', 1),
 ('35481', 1),
 ('1', 1),
 ('34272', 1),
 ('6568', 1),
 ('29513', 1),
 ('19799', 1),
 ('43406', 1),
 ('6115', 1),
 ('9893', 1),
 ('18065', 1),
 ('47442', 1),
 ('23087', 1),
 ('11841', 1),
 ('13829', 1),
 ('21162', 1),
 ('14912', 1),
 ('29490', 1),
 ('7329', 1),
 ('27481', 1),
 ('23486', 1),
 ('22506', 1),
 ('1559', 1),
 ('1937', 1),
 ('2658', 1),
 ('24352', 1),
 ('23924', 1),
 ('6214', 1),
 ('1085', 1),
 ('14778', 1),
 ('31223', 1),
 ('2662', 1),
 ('15369', 1),
 ('36683', 1),
 ('25405', 1),
 ('15750', 1),
 ('41003', 1),
 ('45019', 1),
 ('27299', 1),
 ('41827', 1),
 ('27343', 1),
 ('1694', 1),
 ('6030', 1),
 ('27530', 1),
 ('43250', 1),
 ('32126', 1),
 ('10411', 1),
 ('27522', 1),
 ('43627', 1),
 ('25178', 1),
 ('38696', 1),
 ('6186', 1),
 ('34536', 1),
 ('43224', 1),
 ('43592', 1),
 ('3952', 1),
 ('31236', 1),
 ('4233', 1),
 ('27503', 1),
 ('39547', 1),
 ('19034', 1),
 ('1667', 1),
 ('29817', 1),
 ('7522', 1),
 ('43223', 1),
 ('27532', 1),
 ('26872', 1),
 ('32063', 1),
 ('29528', 1),
 ('40043', 1),
 ('27480', 1),
 ('24627', 1),
 ('49985', 1),
 ('32956', 1),
 ('27561', 1),
 ('9639', 1),
 ('29561', 1),
 ('43602', 1),
 ('27525', 1),
 ('44385', 1),
 ('29767', 1),
 ('43401', 1),
 ('27679', 1),
 ('37872', 1),
 ('2062', 1),
 ('49946', 1),
 ('41688', 1),
 ('10280', 1),
 ('43148', 1),
 ('1230', 1),
 ('46868', 1),
 ('21785', 1),
 ('43242', 1),
 ('37226', 1),
 ('20575', 1),
 ('42488', 1),
 ('1586', 1),
 ('6574', 1),
 ('17328', 1),
 ('34606', 1),
 ('11897', 1),
 ('32631', 1),
 ('43348', 1),
 ('2645', 1),
 ('27489', 1),
 ('10151', 1),
 ('14009', 1),
 ('30257', 1),
 ('6452', 1),
 ('1950', 1),
 ('19060', 1),
 ('44899', 1),
 ('9170', 1),
 ('27520', 1),
 ('120', 1),
 ('29229', 1),
 ('18190', 1),
 ('1602', 1),
 ('27482', 1),
 ('44092', 1),
 ('12352', 1),
 ('18924', 1),
 ('27736', 1),
 ('43861', 1),
 ('10

In [27]:
'''
take_top_10 returns top 10
friend recommendations for each person
'''

In [28]:
def take_top_10(x):
  if len(x[1])<10:
    return (x[0],x[1])
  else:
    return (x[0],x[1][:10])

In [29]:
recommend = friend_recom.map(take_top_10)

In [30]:
recommend.take(2)

Out[ 48 ]: 
[('30969',
 [('45468', 3),
 ('22633', 3),
 ('29594', 3),
 ('29592', 3),
 ('23799', 2),
 ('45515', 2),
 ('17970', 2),
 ('29597', 2),
 ('31015', 2),
 ('7466', 2)]),
 ('20429',
 [('20401', 1),
 ('20423', 1),
 ('20419', 1),
 ('20414', 1),
 ('20400', 1),
 ('20422', 1),
 ('20441', 1),
 ('20410', 1),
 ('20399', 1),
 ('20413', 1)])]